In [8]:
using TSSOS
using DynamicPolynomials
using CSV
using Random
using Statistics
using DataFrames
using DelimitedFiles

In [9]:
ts = readdlm( "stock-market.txt" );
pred=1

1

In [14]:
T=20
Y=observation(T+pred,ts)
parameter_estimation2(Y[1:T])

***************************TSSOS***************************
TSSOS is launching...
Starting to compute the block structure...
------------------------------------------------------
The sizes of PSD blocks:
[3, 2, 1]
[20, 60, 1]
------------------------------------------------------
Obtained the block structure in 0.0052675 seconds. The maximal size of blocks is 3.
Assembling the SDP...
There are 3486 affine constraints.
SDP assembling time: 1.0137763 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3486            
  Cones                  : 0               
  Scalar variables       : 82              
  Matrix variables       : 161             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed con

30.84300263520486

In [15]:
Y[T+1:T+pred]

1-element Vector{Float64}:
 30.5235

In [10]:
function parameter_estimation2(Y)
    # training process
    # use Y to learn the system parameters in f[t+1] = Fdash*f[t] + p[t]
    # output Fdash and p[1:T]
    
    T=length(Y)
    
    @ncpolyvar G p[1:T] q[1:T-1] f[1:T+1] m[1:T+1]; # all variables are assumed to be nc
    var=vcat(m,f,p,q,G);

    # constraints
    ine1 = [f[i] - m[i] - p[i] for i in 1:T];
    ine2 = [- f[i] + m[i] + p[i] for i in 1:T];
    ine3 = [m[i+1] - G*m[i] - q[i] for i in 1:T-1];
    ine4 = [-m[i+1] + G*m[i] + q[i] for i in 1:T-1];
    ine5 = [m[T+1] - G*m[T],-m[T+1] + G*m[T]]
    #ine6 = [f[T+1] - m[T+1],-f[T+1] + F*m[T+1]]

    #objective
    obj=sum((Y[i]-f[i])^2 for i in 1:T)+ 0.01*sum(p[i]^2 for i in 1:T) +0.01*sum(q[i]^2 for i in 1:T-1) #+ p[T+1]^2 + q[T]^2 #+ 0.0001*sum(p[i]^2 for i in 1:T); #+0.1*q[i]^2

    # pop
    pop=vcat(obj,ine1,ine2,ine3,ine4,ine5); #,ine5,ine6

    # solve model 
    opt,sol,data=tssos_first(pop,var,1,TS="MD",solution=true);
    print(sol)
    return sol[T+1]  #+mean(sol[end-T+1:end])
    #sol[1],sol[end-T+1:end]
    #return abs(sol[1]*sol[T+2] - Y[T+2])/Y[T+2] # nrmse
    
end

parameter_estimation2 (generic function with 1 method)

In [50]:
# read observations
# the stock-market.txt is generated by
"""
# Load stock-market data
load_path = 'setting6.mat'
load_data = sio.loadmat(load_path)
seq=flatten(load_data['seq_d0'].tolist())
"""
ts = readdlm( "stock-market.txt" );

T=20
pred=1

opt=DataFrame(time=[],pred=[],rmse=[])
for s in 1:100
    Y=observation(T+pred,ts,s);
    #Y=copy(ts[start:T+pred+start-1])
    Y_predict=parameter_estimation2(Y[1:T]);
    opt_tem=rmse(Y_predict,Y[T+1:T+pred]);
    push!(opt,(s,Y_predict,opt_tem))
end

***************************TSSOS***************************
TSSOS is launching...
Starting to compute the block structure...
------------------------------------------------------
The sizes of PSD blocks:
[3, 2, 1]
[20, 60, 1]
------------------------------------------------------
Obtained the block structure in 0.0248847 seconds. The maximal size of blocks is 3.
Assembling the SDP...
There are 3486 affine constraints.
SDP assembling time: 0.6914166 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3486            
  Cones                  : 0               
  Scalar variables       : 82              
  Matrix variables       : 161             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed con

0   1.9e+04  1.0e+00  1.0e+00  0.00e+00   0.000000000e+00   0.000000000e+00   1.0e+00  3.80  
1   4.6e+02  2.4e-02  1.4e-01  -9.85e-01  5.241990654e-01   -2.967956436e+01  2.4e-02  15.56 
2   1.8e+02  9.7e-03  6.0e-02  -5.04e-01  1.221099086e+02   8.410966607e+01   9.7e-03  27.50 
3   1.2e+02  6.5e-03  3.8e-02  1.49e-02   7.750652285e+03   7.716985144e+03   6.5e-03  39.41 
4   2.7e+01  1.4e-03  3.5e-03  2.66e-01   1.550811917e+04   1.550230802e+04   1.4e-03  51.28 
5   7.2e+00  3.8e-04  7.2e-04  2.93e-01   1.510256193e+04   1.509895378e+04   3.8e-04  63.28 
6   1.7e+00  8.7e-05  1.8e-04  -1.79e-01  1.299980953e+04   1.299571100e+04   8.7e-05  75.09 
7   4.7e-01  2.5e-05  4.3e-05  1.67e-01   9.764204717e+03   9.761136409e+03   2.5e-05  87.02 
8   1.3e-01  6.7e-06  1.0e-05  2.43e-02   6.087003632e+03   6.084735557e+03   6.7e-06  98.86 
9   3.8e-02  2.0e-06  2.3e-06  6.08e-01   2.989017549e+03   2.987718799e+03   2.0e-06  110.52
10  7.9e-03  4.2e-07  2.4e-07  9.65e-01   8.194756844e+02   

------------------------------------------------------
Obtained the block structure in 0.0989296 seconds. The maximal size of blocks is 3.
Assembling the SDP...
There are 3486 affine constraints.
SDP assembling time: 0.8199394 seconds.
Solving the SDP...
12  3.2e-04  1.7e-08  1.3e-09  1.26e+00   2.829250646e+01   2.828640894e+01   1.7e-08  151.00
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3486            
  Cones                  : 0               
  Scalar variables       : 82              
  Matrix variables       : 161             
  Integer variables      : 0               

Optimizer started.


error in running finalizer: ErrorException("task switch not allowed from inside gc finalizer")
jl_error at /cygdrive/c/buildbot/worker/package_win64/build/src\rtutils.c:41
jl_switch at /cygdrive/c/buildbot/worker/package_win64/build/src\task.c:502
try_yieldto at .\task.jl:700
wait at .\task.jl:770
uv_write at .\stream.jl:992
unsafe_write at .\stream.jl:1064
unsafe_write at .\io.jl:361 [inlined]
write at .\strings\io.jl:185 [inlined]
print at .\strings\io.jl:187
unknown function (ip: 000000000233db44)
print at .\coreio.jl:3
#16 at C:\Users\v-quazhou\.julia\packages\MosekTools\sppJY\src\MosekTools.jl:363
msk_stream_callback_wrapper at C:\Users\v-quazhou\.julia\packages\Mosek\Ue6iA\src\msk_callback.jl:7
unknown function (ip: 000000000230115d)
MSK_unittestmain at C:\Program Files\Mosek\9.3\tools\platform\win64x86\bin\mosek64_9_3.dll (unknown line)
MSK_optimizetrm at C:\Program Files\Mosek\9.3\tools\platform\win64x86\bin\mosek64_9_3.dll (unknown line)
MSK_unittestmain at C:\Program Files\Mo

Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3486            
  Cones                  : 0               
  Scalar variables       : 82              
  Matrix variables       : 161             
  Integer variables      : 0               

Optimizer  - threads                : 6               
Optimizer  - solved problem         : the primal      
Optimizer  - Constraints            : 3486
Optimizer  

...

1   5.8e+02  2.4e-02  1.4e-01  -9.88e-01  4.409288573e-01   -3.129648088e+01  2.4e-02  15.86 
2   1.9e+02  8.0e-03  5.6e-02  -5.87e-01  1.046007537e+02   5.526967795e+01   8.0e-03  28.03 
3   1.5e+02  6.5e-03  4.4e-02  -1.49e-03  2.436240675e+03   2.391122297e+03   6.5e-03  39.98 
4   7.9e+01  3.3e-03  1.9e-02  6.84e-02   1.409215397e+04   1.405970028e+04   3.3e-03  51.78 
5   1.6e+01  6.6e-04  2.0e-03  5.57e-01   1.934979061e+04   1.934102001e+04   6.6e-04  63.75 
6   5.6e+00  2.3e-04  5.7e-04  8.52e-01   1.840738568e+04   1.840144999e+04   2.3e-04  75.66 
7   3.0e+00  1.3e-04  4.9e-04  2.97e-01   1.699352524e+04   1.697850665e+04   1.3e-04  87.58 
8   9.0e-01  3.8e-05  7.0e-05  1.39e+00   1.301011608e+04   1.300678143e+04   3.8e-05  99.45 
9   1.4e-01  5.8e-06  1.1e-05  -6.96e-02  7.813025647e+03   7.809179866e+03   5.8e-06  111.36
10  2.5e-02  1.1e-06  1.1e-06  1.30e-01   2.801932478e+03   2.800908247e+03   1.1e-06  123.38
11  1.6e-02  6.7e-07  3.5e-07  7.81e-01   1.520230522e+03   

There are 3486 affine constraints.
SDP assembling time: 1.173587 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 3486            
  Cones                  : 0               
  Scalar variables       : 82              
  Matrix variables       : 161             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.01    
Problem
  Name                   :                 


0   2.4e+04  1.0e+00  1.0e+00  0.00e+00   0.000000000e+00   0.000000000e+00   1.0e+00  3.84  
1   5.8e+02  2.4e-02  1.4e-01  -9.88e-01  4.389704536e-01   -3.133450147e+01  2.4e-02  15.61 
2   1.9e+02  8.0e-03  5.6e-02  -5.89e-01  1.042494391e+02   5.459942174e+01   8.0e-03  27.59 
3   1.6e+02  6.5e-03  4.4e-02  -2.03e-03  2.382865738e+03   2.337430012e+03   6.5e-03  39.28 
4   7.9e+01  3.3e-03  1.9e-02  6.54e-02   1.419318020e+04   1.416032307e+04   3.3e-03  51.16 
5   1.6e+01  6.6e-04  2.0e-03  5.45e-01   1.946744273e+04   1.945853726e+04   6.6e-04  63.47 
6   5.7e+00  2.4e-04  5.8e-04  8.71e-01   1.855088264e+04   1.854494827e+04   2.4e-04  75.72 
7   3.0e+00  1.3e-04  4.8e-04  2.36e-01   1.711678816e+04   1.710206695e+04   1.3e-04  87.63 
8   9.1e-01  3.8e-05  6.9e-05  1.37e+00   1.312374377e+04   1.312049779e+04   3.8e-05  99.66 
9   1.3e-01  5.5e-06  1.1e-05  -8.24e-02  7.755441442e+03   7.751584268e+03   5.5e-06  111.55
10  4.7e-02  2.0e-06  2.2e-06  1.48e-01   3.956729615e+03   

Excessive output truncated after 524343 bytes.


Problem
  Name                   :                 


In [54]:
CSV.write("stock_TSSOS_d1.csv", opt)

"stock_TSSOS_d1.csv"

In [53]:
opt

,time,pred,rmse
,Any,Any,Any
1,1,31.1972,0.00360887
2,2,31.2437,0.217652
3,3,31.134,0.216403
4,4,31.0033,0.317241
5,5,30.8114,0.273076
6,6,30.6107,0.104938
7,7,30.6583,0.249499
8,8,30.8293,0.118622
9,9,30.7608,0.291475


In [32]:
opt

50-element Vector{Float64}:
 0.31915582882466254
 0.5067156292252193
 0.03324578633905162
 0.1838445819894794
 0.18160609912393966
 0.3153700971368636
 0.03446912691558879
 0.2832462940750844
 0.1271634281960985
 0.5383255704419057
 0.16480192824889173
 0.1586156915992305
 0.17229103928545442
 ⋮
 0.1271634281960985
 0.2540214167075874
 0.2187073566154325
 0.4614037716308239
 0.18160609912393966
 0.12628966064029612
 0.36372647824434523
 0.17901402833597757
 0.29570514359345523
 0.29570514359345523
 0.2540214167075874
 0.25435814510831634

In [46]:
function observation(T,ts,start)
    # randomly select a trajectory of time series
    return copy(ts[start:start+T-1])
end

observation (generic function with 2 methods)

In [49]:
function rmse(Y_predict, Y_true)
    # compare predicted Y_[T+1：T+pred] with actual value of predicted Y_[T+1：T+pred]
    # calcluate rmse: sqrt( sum((Y-f)^2)/N) 
    Y=copy(Y_true[1:length(Y_predict)])
    return  abs(Y[1]-Y_predict[1])
    #1-sum( abs(Y[i]-Y_predict[i]) for i in 1:length(Y_predict) )/ sum(abs(Y[i]-mean(Y)) for i in 1:length(Y_predict) )
end

rmse (generic function with 1 method)

In [47]:
function prediction(inital,para,T)
    # use the learned system parameters and Y[T] to predict Y_[T+1：T+pred]
    # output predicted Y_[T+1：T+pred]
    Y_predict=Float64[]
    push!(Y_predict,inital*para[1])
    if T >= 2
        for i in 2:T
            push!(Y_predict,Y_predict[end]*para[1]) #+mean(para[2]))
        end
    end
    return Y_predict
end

prediction (generic function with 1 method)

In [62]:
function parameter_estimation1(Y)
    # training process
    # use Y to learn the system parameters in f[t+1] = Fdash*f[t] + p[t]
    # output Fdash and p[1:T]
    
    T=length(Y)-1
    
    @ncpolyvar Fdash p[1:T] f[1:T+2]; # all varaibles are assumed to be nc
    var=vcat(Fdash,f,p);

    # constraints
    ine1 = [f[i+1] - p[i] - Fdash*f[i] for i in 1:T];
    ine2 = [- f[i+1] + p[i] + Fdash*f[i] for i in 1:T];
    ine3 = [f[T+2] - Fdash*f[T+1]]
    ine4 = [-f[T+2] + Fdash*f[T+1]]

    #objective
    obj=sum((Y[i]-f[i])^2 for i in 1:T+1) + sum(p[i]^2 for i in 1:T);

    # pop
    pop=vcat(obj,ine1,ine2,ine3,ine4);

    # solve model 
    opt,sol,data=tssos_first(pop,var,1,TS="MD",solution=true);
    print(sol)
    return sol[T+3]  #+mean(sol[end-T+1:end])
    #sol[1],sol[end-T+1:end]
    #return abs(sol[1]*sol[T+2] - Y[T+2])/Y[T+2] # nrmse
    
end

parameter_estimation1 (generic function with 1 method)

In [38]:
function parameter_estimation(Y)
    # training process
    # use Y to learn the system parameters in f[t+1] = Fdash*f[t] + p[t]
    # output Fdash and p[1:T]
    
    T=length(Y)-1
    
    @ncpolyvar Fdash p[1:T] f[1:T+1]; # all varaibles are assumed to be nc
    var=vcat(Fdash,f,p);

    # constraints
    ine1 = [f[i+1] - p[i] - Fdash*f[i] for i in 1:T];
    ine2 = [- f[i+1] + p[i] + Fdash*f[i] for i in 1:T];

    #objective
    obj=sum((Y[i]-f[i])^2 for i in 1:T+1);

    # pop
    pop=vcat(obj,ine1,ine2);

    # solve model 
    opt,sol,data=tssos_first(pop,var,2,TS="MD",solution=true);
    print(sol)
    return sol[1],sol[end-T+1:end]
    #return abs(sol[1]*sol[T+2] - Y[T+2])/Y[T+2] # nrmse
    
end

parameter_estimation (generic function with 1 method)

In [ ]:
opt # Benchmark with extra term in obj  and state-space

In [66]:
opt # Benchmark with extra term in obj

3-element Array{Float64,1}:
 0.6025681319810872
 0.5306212983910328
 0.6272917502641597

In [56]:
opt # Benchmark no extra term in obj

30-element Array{Float64,1}:
 0.20493633878830053
 0.9508318273950849
 1.1067277351584721
 0.7250721166269471
 0.22866002485903536
 0.08768005331801021
 0.3865505163066736
 0.8371891913525857
 0.8679356521379518
 0.9508318273950849
 0.2902265279848919
 0.5349447141394634
 1.53028389638569
 ⋮
 0.5392562789066204
 0.08684888170039073
 1.3986490885563114
 0.15488534013059763
 0.81188261490502
 0.5753328941804945
 1.65058746764403
 0.6609721403599949
 1.1722957976485304
 0.35225763055220227
 1.1722957976485304
 2.8327631184976774

In [29]:
# read observations
# the stock-market.txt is generated by
"""
# Load stock-market data
load_path = 'setting6.mat'
load_data = sio.loadmat(load_path)
seq=flatten(load_data['seq_d0'].tolist())
"""
ts = readdlm( "stock-market.txt" );

T=30
pred=1

meanO=Float64[]
stdO=Float64[]

#meanT=Float64[]
#stdT=Float64[]

opt=Float64[]
for r in 1:3
    Y=observation(T+pred,ts);
    #Y=copy(ts[start:T+pred+start-1])
    para=parameter_estimation(Y[1:T]);
    #fill!(para[2], 0)
    Y_predict=prediction(Y[T],para,pred);
    opt_tem=rmse(Y_predict,Y[T+1:T+pred]);
    print(opt_tem)
    push!(opt,copy(opt_tem))
end

push!(meanO,mean(copy(opt)))
push!(stdO,std(copy(opt)))

R=convert(DataFrame,hcat(meanO,stdO))
CSV.write("stock_TSSOS.csv",R, writeheader=false)
writedlm( "stock_TSSOS_opt.csv", opt, ',')

------------------------------------------------------
The sizes of blocks:
[3, 2]
[29, 30]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1891            
  Cones                  : 0               
  Scalar variables       : 59              
  Matrix variables       : 118             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.02   

10  6.7e-02  1.9e-06  2.9e-06  6.67e-01   3.057358929e+03   3.054969750e+03   1.9e-06  37.50 
11  7.8e-03  2.2e-07  1.3e-07  9.59e-01   4.336566434e+02   4.333258210e+02   2.2e-07  41.80 
12  7.3e-05  2.1e-09  1.1e-10  9.62e-01   4.359811588e+00   4.356951204e+00   2.1e-09  45.33 
13  6.8e-07  2.0e-11  9.4e-14  1.00e+00   3.507345034e-02   3.505009439e-02   1.8e-11  48.81 
14  2.9e-08  9.2e-10  1.0e-17  1.00e+00   8.039652892e-05   8.034474844e-05   4.0e-14  52.45 
15  2.2e-07  8.1e-07  1.3e-19  1.00e+00   2.122941326e-06   2.122686903e-06   1.9e-16  73.38 
Optimizer terminated. Time: 73.39   

optimum = 2.122941326102815e-6
[1.029192930815133, 33.91832523732298, 33.854199624031764, 34.26814319478198, 34.0414025924366, 34.44550418531527, 34.84947485091373, 35.52458998428701, 34.736159698688525, 35.24865065020481, 34.94801443574965, 34.997322417285275, 35.135345076833055, 34.775611513433816, 34.450419868791755, 34.583385860984976, 35.021874284469185, 34.78546316297564, 34.24337196615904

In [31]:
opt

3-element Array{Float64,1}:
 1.052557715727822
 0.5880377045388769
 1.520180822607827

# The end

In [170]:
start=100
T=30
pred=1

Y=observation(T+pred,ts);
Y=copy(ts[start:T+pred+start-1])
para=parameter_estimation(Y[1:T]);
#fill!(para[2], 0)
Y_predict=prediction(Y[T],para,pred);
opt_tem=mape(Y_predict,Y[T+1:T+pred])

------------------------------------------------------
The sizes of blocks:
[3, 2]
[29, 30]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1891            
  Cones                  : 0               
  Scalar variables       : 59              
  Matrix variables       : 118             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.02   

0.00819129124945271

In [5]:
T=20
Y=copy(ts[1:1+T+1])
# set nc operators
@ncpolyvar Fdash p[1:T] f[1:T+1]; # all varaibles are assumed to be nc
var=vcat(Fdash,f,p);

# constraints
ine1 = [f[i+1]- Fdash*f[i] - p[i] for i in 1:T];
ine2 = [- f[i+1] + Fdash*f[i] + p[i] for i in 1:T];

#objective
obj=sum((Y[i]-f[i])^2 for i in 1:T+1);

# pop
pop=vcat(obj,ine1,ine2);

# solve model 
opt,sol,data=tssos_first(pop,var,1,TS="MD",solution=true);

------------------------------------------------------
The sizes of blocks:
[3, 2]
[20, 21]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 946             
  Cones                  : 0               
  Scalar variables       : 41              
  Matrix variables       : 82              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00   

In [9]:
meanO=Float64[]
stdO=Float64[]

for t in 15:16
    opt=Float64[]
    for r in 1:3
        opt_tem=ncpop(t);
        push!(opt,opt_tem)
    end
    
    push!(meanO,mean(copy(opt)))
    push!(stdO,std(copy(opt)))
    
end

------------------------------------------------------
The clique sizes of varibles:
[3, 2, 1]
[14, 2, 45]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 158             
  Cones                  : 0               
  Scalar variables       : 62              
  Matrix variables       : 151             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated

optimum = -27.416717984066867
------------------------------------------------------
The clique sizes of varibles:
[3, 2, 1]
[15, 2, 48]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 168             
  Cones                  : 0               
  Scalar variables       : 66              
  Matrix variables       : 161             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0     

------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 168             
  Cones                  : 0               
  Scalar variables       : 66              
  Matrix variables       : 161             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective sense        : max        

In [6]:
meanR=Float64[]
stdR=Float64[]

for t in 15:16
    Re=Float64[]
    for r in 1:3
        timeR=@elapsed ncpop(t);
        push!(Re,timeR)
    end
    
    push!(meanR,mean(copy(Re)))
    push!(stdR,std(copy(Re)))
    
end

------------------------------------------------------
The clique sizes of varibles:
[3, 2, 1]
[14, 2, 45]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 158             
  Cones                  : 0               
  Scalar variables       : 62              
  Matrix variables       : 151             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated

11  3.4e-06  2.2e-10  6.8e-07  -9.99e-01  -2.622109146e+12  -2.648694378e+12  4.4e-12  0.06  
12  6.1e-09  1.3e-08  4.9e-08  -1.00e+00  -1.263354040e+15  -1.276162423e+15  9.1e-15  0.09  
13  5.2e-09  1.7e-08  3.7e-08  -1.00e+00  -1.301829435e+15  -1.315027337e+15  8.9e-15  0.11  
14  2.0e-09  4.8e-08  5.7e-09  -1.00e+00  -2.119347980e+16  -2.140811960e+16  5.4e-16  0.11  
Optimizer terminated. Time: 0.11    

termination status: SLOW_PROGRESS
solution status: NO_SOLUTION
optimum = -27.416717984066867
------------------------------------------------------
The clique sizes of varibles:
[3, 2, 1]
[15, 2, 48]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 168             
  Cones                  : 0               
  Scalar variables       : 66              
  Matrix variables       : 161           

optimum = -29.20533287143395
------------------------------------------------------
The clique sizes of varibles:
[3, 2, 1]
[15, 2, 48]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 168             
  Cones                  : 0               
  Scalar variables       : 66              
  Matrix variables       : 161             
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0      

In [4]:
T=10

# set nc operators
@ncpolyvar G Fdash m[1:T+1] q[1:T] p[1:T] f[1:T]; # all varaibles are assumed to be nc
var=vcat(G,Fdash,f,m,p,q);

# constraints
ine1 = [f[i] - p[i] - Fdash*m[i+1] for i in 1:T];
ine2 = [- f[i] + p[i] + Fdash*m[i+1] for i in 1:T];
ine3 = [m[i+1] - q[i] - G*m[i] for i in 1:T];
ine4 = [- m[i+1] + q[i] + G*m[i] for i in 1:T];

#objective
obj=sum((Y[i]-f[i])^2 for i in 1:T) #+ sum(p[i]^2+q[i]^2 for i in 1:T);

# pop
pop=vcat(obj,ine1,ine2,ine3,ine4);

# solve model 
opt,sol,data=tssos_first(pop,var,1,TS="MD",solution=true);
#opt,sol,data = tssos_higher!(data, TS="MD");

------------------------------------------------------
The sizes of blocks:
[4, 3, 2]
[9, 1, 31]
------------------------------------------------------
Problem
  Name                   :                 
  Objective sense        : max             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 990             
  Cones                  : 0               
  Scalar variables       : 41              
  Matrix variables       : 82              
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.01            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.